<a href="https://colab.research.google.com/github/gnicolasgutierrez/Coder_DE_Weather-data-redshift/blob/main/coder_de_weather_data_redshift.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#**Proyecto de Extracción y Análisis de Datos Climáticos**

**Objetivo del Proyecto**

El objetivo de este proyecto es desarrollar un proceso automatizado para la extracción, transformación y carga (ETL) de datos climáticos utilizando la API de OpenWeatherMap. Los datos se obtendrán diariamente y se almacenarán en una base de datos en Amazon Redshift, permitiendo un análisis detallado de las condiciones climáticas de diversas ciudades.

**Descripción del Proyecto**

En este proyecto, se busca:

Automatizar la extracción de datos climáticos: Utilizando la API de OpenWeatherMap, se obtendrán datos climáticos de múltiples ciudades. Estos datos incluirán variables como la temperatura, humedad, presión atmosférica, y descripción del clima.

**Transformar los datos:**

Los datos extraídos se procesarán para asegurar que estén en el formato adecuado antes de su almacenamiento. Esto incluye la normalización de unidades de medida y el manejo de datos faltantes o erróneos.

**Cargar los datos en Redshift:**

 Los datos procesados se cargarán en una tabla de Amazon Redshift. Esta tabla se ha diseñado para incluir una columna temporal que permita controlar y rastrear las ingestas de datos.

**Flujo de Trabajo**

**Extracción:**
Se realiza una solicitud a la API de OpenWeatherMap para obtener los datos climáticos de una ciudad específica.

**Transformación**: Los datos recibidos se procesan para asegurarse de que estén en el formato correcto y se añade una marca de tiempo para el control de ingestas.

**Carga:** Los datos procesados se insertan en una tabla de Amazon Redshift.

**Consulta y análisis:** Los datos almacenados en Redshift se consultan y analizan para extraer información relevante y generar reportes.

**Tecnologías Utilizadas**

**Python:** Lenguaje de programación utilizado para escribir los scripts de ETL.

**psycopg2:** Biblioteca utilizada para conectarse a Amazon Redshift desde Python.

**OpenWeatherMap API:** Fuente de datos climáticos.

**Amazon Redshift:** Base de datos utilizada para almacenar y gestionar los datos climáticos.


#**Consultar datos de la API y preparar los datos**

In [ ]:
import requests
import json
from datetime import datetime

# Configuración de la API
API_KEY = 'f69660d66b33c4f56ca1513daed236da'  # Reemplaza con tu API Key
CITIES = ['Tunuyán', 'Mendoza', 'Buenos Aires', 'Córdoba', 'Rosario', 'La Plata', 'Mar del Plata', 'San Miguel de Tucumán', 'Salta', 'Santa Fe']
API_URL_TEMPLATE = 'http://api.openweathermap.org/data/2.5/weather?q={city}&appid={api_key}'

weather_data_list = []

for city in CITIES:
    api_url = API_URL_TEMPLATE.format(city=city, api_key=API_KEY)
    response = requests.get(api_url)
    weather_data = response.json()

    # Verificación de datos
    if 'main' in weather_data:
        # Transformación de datos
        data = {
            'city': city,
            'temperature': weather_data['main']['temp']-273.15,
            'humidity': weather_data['main']['humidity'],
            'pressure': weather_data['main']['pressure'],
            'weather': weather_data['weather'][0]['description'],
            'timestamp': datetime.now()
        }
        weather_data_list.append(data)
    else:
        print(f"Error en la respuesta de la API para {city}: {weather_data}")

# Imprimir los datos recopilados
for data in weather_data_list:
    print(data)



{'city': 'Tunuyán', 'temperature': 6.640000000000043, 'humidity': 49, 'pressure': 1028, 'weather': 'clear sky', 'timestamp': datetime.datetime(2024, 7, 28, 13, 20, 39, 240275)}
{'city': 'Mendoza', 'temperature': 9.740000000000009, 'humidity': 32, 'pressure': 1026, 'weather': 'scattered clouds', 'timestamp': datetime.datetime(2024, 7, 28, 13, 20, 39, 309116)}
{'city': 'Buenos Aires', 'temperature': 10.580000000000041, 'humidity': 52, 'pressure': 1027, 'weather': 'clear sky', 'timestamp': datetime.datetime(2024, 7, 28, 13, 20, 39, 346237)}
{'city': 'Córdoba', 'temperature': 10.510000000000048, 'humidity': 61, 'pressure': 1026, 'weather': 'clear sky', 'timestamp': datetime.datetime(2024, 7, 28, 13, 20, 39, 387505)}
{'city': 'Rosario', 'temperature': 10.620000000000005, 'humidity': 22, 'pressure': 1025, 'weather': 'clear sky', 'timestamp': datetime.datetime(2024, 7, 28, 13, 20, 39, 428155)}
{'city': 'La Plata', 'temperature': 10.450000000000045, 'humidity': 51, 'pressure': 1027, 'weather':

#**Conectar a Redshift, crear la tabla y definir funciones de inserción**

In [ ]:
import psycopg2

# Configuración de Redshift
REDSHIFT_CONN = {
    'dbname': 'data-engineer-database',
    'user': 'ngutierrez_coderhouse',
    'password': 'L4et1wP7eZ',
    'host': 'data-engineer-cluster.cyhh5bfevlmn.us-east-1.redshift.amazonaws.com',
    'port': '5439'
}

def connect_to_redshift():
    try:
        conn = psycopg2.connect(
            dbname=REDSHIFT_CONN['dbname'],
            user=REDSHIFT_CONN['user'],
            password=REDSHIFT_CONN['password'],
            host=REDSHIFT_CONN['host'],
            port=REDSHIFT_CONN['port']
        )
        print("Conexión exitosa a Redshift")
        return conn
    except psycopg2.OperationalError as e:
        print(f"Error al conectar a Redshift: {e}")

def create_table(conn):
    cursor = conn.cursor()
    create_table_query = """
        CREATE TABLE IF NOT EXISTS weather_data (
            city VARCHAR(50),
            temperature FLOAT,
            humidity INT,
            pressure INT,
            weather VARCHAR(100),
            timestamp TIMESTAMP,
            ingestion_time TIMESTAMP DEFAULT GETDATE()
        )
    """
    cursor.execute(create_table_query)
    conn.commit()
    cursor.close()

def insert_data(conn, data):
    cursor = conn.cursor()
    insert_query = """
        INSERT INTO weather_data (city, temperature, humidity, pressure, weather, timestamp)
        VALUES (%s, %s, %s, %s, %s, %s)
    """
    cursor.execute(insert_query, (data['city'], data['temperature'], data['humidity'], data['pressure'], data['weather'], data['timestamp']))
    conn.commit()
    cursor.close()



#**Insertar datos en Redshift**

In [ ]:
# Uso de las funciones definidas
conn = connect_to_redshift()
if conn:
    create_table(conn)
    # Inserta los datos de todas las ciudades
    for data in weather_data_list:
        insert_data(conn, data)
    conn.close()

print("Datos insertados correctamente en Redshift.")





Conexión exitosa a Redshift
Datos insertados correctamente en Redshift.


#**Consultar los datos desde Redshift**





In [ ]:
def query_data(conn):
    cursor = conn.cursor()
    query = "SELECT * FROM weather_data"
    cursor.execute(query)
    rows = cursor.fetchall()
    for row in rows:
        print(row)
    cursor.close()

# Conexión y consulta
conn = connect_to_redshift()
if conn:
    query_data(conn)
    conn.close()



Conexión exitosa a Redshift
('Tunuyán', 6.08000000000004, 49, 1028, 'clear sky', datetime.datetime(2024, 7, 28, 12, 57, 52, 303161))
('Mendoza', 8.74000000000001, 40, 1026, 'scattered clouds', datetime.datetime(2024, 7, 28, 12, 57, 52, 349236))
('Buenos Aires', 9.77000000000004, 58, 1027, 'clear sky', datetime.datetime(2024, 7, 28, 12, 57, 52, 388559))
('Córdoba', 9.89000000000004, 65, 1026, 'clear sky', datetime.datetime(2024, 7, 28, 12, 57, 52, 431675))
('Rosario', 8.51000000000005, 10, 1025, 'few clouds', datetime.datetime(2024, 7, 28, 12, 57, 52, 473699))
('La Plata', 9.34000000000003, 51, 1027, 'few clouds', datetime.datetime(2024, 7, 28, 12, 57, 52, 518991))
('Mar del Plata', 8.02000000000004, 53, 1027, 'clear sky', datetime.datetime(2024, 7, 28, 12, 57, 52, 558026))
('San Miguel de Tucumán', 15.94, 52, 1021, 'overcast clouds', datetime.datetime(2024, 7, 28, 12, 57, 52, 603476))
('Salta', 15.3200000000001, 74, 1020, 'overcast clouds', datetime.datetime(2024, 7, 28, 12, 57, 52, 64

#**Restablecer tabla**

In [ ]:
def clear_table(conn):
    cursor = conn.cursor()
    clear_table_query = "DELETE FROM weather_data"
    cursor.execute(clear_table_query)
    conn.commit()
    cursor.close()
    print("Tabla weather_data limpiada correctamente.")

# Conexión y limpieza de la tabla
conn = connect_to_redshift()
if conn:
    clear_table(conn)
    conn.close()


Conexión exitosa a Redshift
Tabla weather_data limpiada correctamente.
